In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, col, regexp_replace, split

try:
    spark = SparkSession.builder \
        .appName("Proyecto SparkSQL") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.7.4") \
        .config("spark.driver.extraClassPath", "/path/to/postgresql-42.7.4.jar") \
        .config("spark.executor.extraClassPath", "/path/to/postgresql-42.7.4.jar") \
        .getOrCreate()
    print("SparkSession creada:", spark.version)
except Exception as e:
    print("Error al crear SparkSession:", str(e))

def clean_district_column(df, column_name):
    """
    Elimina espacios en blanco extras y normaliza el texto en una columna
    
    Args:
        df: DataFrame de Spark
        column_name: Nombre de la columna a limpiar
        
    Returns:
        DataFrame con la columna limpia
    """
    # Elimina espacios en blanco múltiples y normaliza
    cleaned_df = df.withColumn(
        column_name,
        trim(regexp_replace(col(column_name), "\\s+", " "))
    )
    
    return cleaned_df

def extract_district_from_compound(df):
    """
    Extrae el distrito de la columna compuesta 'Provincia, cantón y distrito'
    y limpia los espacios
    
    Args:
        df: DataFrame de INEC
        
    Returns:
        DataFrame con nueva columna 'distrito' limpia
    """
    # Divide la columna por comas y toma el último elemento (distrito)
    df_with_district = df.withColumn(
        'distrito',
        trim(split(col('Provincia, cantón y distrito'), ',').getItem(2))
    )
    
    # Limpia la columna distrito
    cleaned_df = clean_district_column(df_with_district, 'distrito')
    
    return cleaned_df

# Uso:
# Para OIJ.csv:
# df_oij_clean = clean_district_column(df_oij, 'Distrito')

# Para INEC.csv:
# df_inec_clean = extract_district_from_compound(df_inec)


SparkSession creada: 3.5.3
